In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
import pymysql
import getpass

In [4]:
pw = getpass.getpass()

········


In [5]:
conn = pymysql.connect(
    host='localhost',
    port=3306,
    user='Lena',
    passwd=pw,
    db='bank')

In [6]:
account = pd.read_sql("SELECT * FROM account;", conn)

In [8]:
account

,account_id,district_id,frequency,date
0,1,18,POPLATEK MESICNE,950324
1,2,1,POPLATEK MESICNE,930226
2,3,5,POPLATEK MESICNE,970707
3,4,12,POPLATEK MESICNE,960221
4,5,15,POPLATEK MESICNE,970530
...,...,...,...,...
4495,11333,8,POPLATEK MESICNE,940526
4496,11349,1,POPLATEK TYDNE,950526
4497,11359,61,POPLATEK MESICNE,941001
4498,11362,67,POPLATEK MESICNE,951014


In [9]:
account.dtypes

account_id      int64
district_id     int64
frequency      object
date            int64
dtype: object

In [24]:
district = pd.read_sql("SELECT A1, A2 FROM district;", conn)

In [25]:
district.head()

,A1,A2
0,1,Hl.m. Praha
1,2,Benesov
2,3,Beroun
3,4,Kladno
4,5,Kolin


In [26]:
district = pd.read_sql("SELECT * FROM district;", conn)

In [27]:
district.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.67,1.85,132,2159,2674
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.95,2.21,111,2824,2813
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.64,5.05,109,5244,5892
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,9307,3.85,4.43,118,2616,3040


In [28]:
distr_cols = {"A1": "district_id",
             "A3": "region"}

district = district.rename(columns=distr_cols)

#or district.rename(columns=distr_cols, inplace=True)

In [29]:
district.head()

,district_id,A2,region,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.67,1.85,132,2159,2674
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.95,2.21,111,2824,2813
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.64,5.05,109,5244,5892
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,9307,3.85,4.43,118,2616,3040


In [30]:
district = district[["district_id", "region"]]

In [31]:
district.head()

,district_id,region
0,1,Prague
1,2,central Bohemia
2,3,central Bohemia
3,4,central Bohemia
4,5,central Bohemia


In [35]:
#join

account_region = account.merge(right=district, on="district_id")
account_region

,account_id,district_id,frequency,date,region
0,1,18,POPLATEK MESICNE,950324,south Bohemia
1,343,18,POPLATEK MESICNE,940426,south Bohemia
2,413,18,POPLATEK MESICNE,960709,south Bohemia
3,431,18,POPLATEK MESICNE,961105,south Bohemia
4,568,18,POPLATEK MESICNE,950328,south Bohemia
...,...,...,...,...,...
4495,4488,33,POPLATEK MESICNE,960424,north Bohemia
4496,4511,33,POPLATEK MESICNE,930926,north Bohemia
4497,4543,33,POPLATEK MESICNE,931008,north Bohemia
4498,4949,33,POPLATEK MESICNE,961014,north Bohemia


In [51]:
#as list
account_region.groupby("region")["account_id"].count()
#as dataframe
account_region.groupby("region")[["account_id"]].count()

,account_id
region,
Prague,554
central Bohemia,574
east Bohemia,544
north Bohemia,457
north Moravia,793
south Bohemia,370
south Moravia,778
west Bohemia,430


In [56]:
accounts_by_region = (account_region
                      .groupby("region")["account_id"]
                      .count()
                      .reset_index()
                      .rename(columns={"account_id": "no_of_accounts"}))


In [57]:
accounts_by_region.head()

,region,no_of_accounts
0,Prague,554
1,central Bohemia,574
2,east Bohemia,544
3,north Bohemia,457
4,north Moravia,793


In [62]:
#to sum how many accounts are in Prague

np.where(account_region["region"] == "Prague", 1, 0).sum()

#to get perecentage of accounts in Prague (12,31%)

np.where(account_region["region"] == "Prague", 1, 0).mean()

0.12311111111111112

In [66]:
accounts_by_region["no_of_accounts"].sum()

4500

In [67]:
accounts_by_region["handicap"]=70

In [70]:
#DO NOT CREATE ACCOUNTS LIKE THIS!!!
#to create new column: ["the_sum_of_it"] that sums up all values based on roes from "no_of_accounts" and "handicap"

accounts_by_region["the_sum_of_it"] = accounts_by_region[["no_of_accounts", "handicap"]].sum(axis=1)

In [71]:
accounts_by_region

,region,no_of_accounts,handicap,the_sum_of_it
0,Prague,554,70,624
1,central Bohemia,574,70,644
2,east Bohemia,544,70,614
3,north Bohemia,457,70,527
4,north Moravia,793,70,863
5,south Bohemia,370,70,440
6,south Moravia,778,70,848
7,west Bohemia,430,70,500


In [73]:
#ALWAYS LIKE THIS!!!


accounts_by_region.assign(the_actual_sum_of_it=accounts_by_region[["no_of_accounts", "handicap"]].sum(axis=1))

,region,no_of_accounts,handicap,the_sum_of_it,the_actual_sum_of_it
0,Prague,554,70,624,624
1,central Bohemia,574,70,644,644
2,east Bohemia,544,70,614,614
3,north Bohemia,457,70,527,527
4,north Moravia,793,70,863,863
5,south Bohemia,370,70,440,440
6,south Moravia,778,70,848,848
7,west Bohemia,430,70,500,500
